In [1933]:
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib_inline.backend_inline import set_matplotlib_formats
from sklearn.neighbors import KNeighborsRegressor as KNN
from sklearn.linear_model import LinearRegression as LR
from sklearn.model_selection import train_test_split
set_matplotlib_formats('svg')
rng = np.random.default_rng()
train = pd.read_csv('/Users/karpov.all/Downloads/train.csv', index_col=0)
test = pd.read_csv('/Users/karpov.all/Downloads/test.csv', index_col=0)
target = train['averageRating']
train = train.drop('averageRating', axis=1)

# countries_list = ['KM', 'MV', 'CM', 'BQ', 'LT', 'JM', 'NG', 'GS', 'BJ', 'BI', 'SB',
#        'FK', 'NE', 'GQ', 'MH', 'AS', 'AL', 'DK', 'SN', 'VA', 'FI', 'NL',
#        'AO', 'NO', 'NZ', 'LR', 'LS', 'CL', 'BV', 'ZW', 'NI', 'SL', 'ME',
#        'SH', 'AF', 'NU', 'AD', 'CU', 'SD', 'AT', 'PS', 'EC', 'MK', 'KH',
#        'PH', 'FJ', 'BD', 'AR', 'AU', 'BA', 'TZ', 'BW', 'HR', 'GE', 'DO',
#        'SJ', 'LU', 'DJ', 'GI', 'KZ', 'TL', 'CO', 'KY', 'SG', 'IM', 'UA',
#        'PE', 'BG', 'CR', 'CG', 'SE', 'LB', 'SI', 'AQ', 'AZ', 'TG', 'BL',
#        'BT', 'SR', 'GH', 'UM', 'UY', 'RU', 'RO', 'BH', 'GY', 'PT', 'IS',
#        'VG', 'ML', 'IR', 'MX', 'KW', 'LC', 'BF', 'SK', 'IO', 'TO', 'MD',
#        'MC', 'EE', 'CY', 'CA', 'KP', 'US', 'JO', 'GD', 'BE', 'SY', 'TH',
#        'PM', 'BN', 'CH', 'BB', 'MS', 'KE', 'HN', 'MP', 'HK', 'ZM', 'OM',
#        'AX', 'IT', 'GU', 'KR', 'YT', 'LA', 'RW', 'LK', 'SZ', 'EH', 'MR',
#        'PK', 'AE', 'GT', 'PY', 'GP', 'ID', 'AG', 'VI', 'SC', 'HM', 'GW',
#        'TK', 'UZ', 'RS', 'TR', 'MA', 'MG', 'VU', 'DM', 'AM', 'TD', 'PN',
#        'IL', 'IE', 'GA', 'BM', 'MM', 'GB', 'VN', 'DZ', 'GM', 'TT', 'BZ',
#        'SA', 'DE', 'KN', 'UG', 'KI', 'NF', 'GR', 'TM', 'CC', 'CI', 'IQ',
#        'SS', 'RE', 'HU', 'IN', 'TN', 'VC', 'TV', 'LV', 'MO', 'QA', 'WF',
#        'GL', 'NC', 'PG', 'JP', 'AI', 'MN', 'MW', 'CN', 'JE', 'CX', 'WS',
#        'CK', 'AW', 'BY', 'NR', 'MF', 'SM', 'SO', 'ST', 'CW', 'PW', 'YE',
#        'FR', 'FM', 'PA', 'TC', 'KG', 'SV', 'CV', 'PF', 'CZ', 'VE', 'BR',
#        'GF', 'MU', 'NP', 'TW', 'PL', 'CF', 'LI', 'EG', 'SX', 'GG', 'PR',
#        'ER', 'BO', 'ZA', 'MY', 'ET', 'BS', 'MQ', 'TJ', 'MZ', 'TF',
#        'FO', 'GN', 'LY', 'CD', 'MT', 'HT', 'ES']

# country_map = {country: index for index, country in enumerate(countries_list)}

genres_list = ['Drama', 'Horror', 'Short', 'Action', 'Animation', 'Crime', 'Thriller',
               'Musical', 'Mystery', 'Biography', 'Documentary', 'Comedy', 'History', 'Family',
               'Romance', 'Western', 'Adventure', 'Fantasy', 'Sci-Fi', 'Sport', 'Music',
               'Adult', 'Reality-TV', 'War', 'News', 'Talk-Show', 'Game-Show', 'Film-Noir']

title_list = ['short', 'movie', 'tvMovie', 'tvEpisode', 'tvSeries', 'tvMiniSeries', 'video',
               'videoGame', 'tvSpecial', 'tvShort']

columns_to_drop = ['primaryTitle', 'mainDirectorBirthYear', 
                   'mainDirectorDeathYear', 'originalTitle', 'mainDirector',
                   'MainDirectorPrimaryProfession', 'mainWriter','mainWriterBirthYear',
                   'mainWriterDeathYear', 'MainWriterPrimaryProfession','firstShownCountry', 'prefVotersCountry',
                   'genres','isAdult','Downloads', 'startYear','Cash', 'titleType']

for genre in genres_list:
    train[genre] = 0

for title in title_list:
    train[title] = 0

for index, row in train.iterrows():
    titles = row['titleType']
    if pd.isna(titles):
        continue
    for title in title_list:
        if title in titles:
            train.at[index, title] = 1
    
for index, row in train.iterrows():
    genres = row['genres']
    if pd.isna(genres):
        continue
    for genre in genres_list:
        if genre in genres:
            train.at[index, genre] = 1
            
train = train.drop(columns=columns_to_drop)
train = train.fillna(train.mean())

# train['firstShownCountry'] = train['firstShownCountry'].map(country_map)
# train['prefVotersCountry'] = train['prefVotersCountry'].map(country_map)


for genre in genres_list:
    test[genre] = 0
    
for title in title_list:
    test[title] = 0


for index, row in test.iterrows():
    genres = row['genres']
    if pd.isna(genres):
        continue
    for genre in genres_list:
        if genre in genres:
            test.at[index, genre] = 1

for index, row in test.iterrows():
    titles = row['titleType']
    if pd.isna(titles):
        continue
    for title in title_list:
        if title in titles:
            test.at[index, title] = 1


test = test.drop(columns=columns_to_drop)
test = test.fillna(test.mean())

# test['firstShownCountry'] = test['firstShownCountry'].map(country_map)
# test['prefVotersCountry'] = test['prefVotersCountry'].map(country_map)


In [1934]:
class Preprocessing:

    def fit(self, X, y):
        self.knn_year = KNN(55).fit(X[['endYear']], y)
        self.knn_min = KNN(55).fit(X[['runtimeMinutes']], y)
        #self.knn_len = KNN(75).fit(X[['titleLength']], y)
        #self.knn_vot = KNN(750).fit(X[['numVotes']], y)
        return self

    def transform(self, X, y=None):
        res = X

        res['1/numVotes'] = 1/(X['numVotes'] ** 1/2)
        res['1/genderVoters'] = X['genderVoters'] ** 1/2

        res['runtimeMinutes^2'] = X['runtimeMinutes'] ** 2
        #res['runtimeMinutes^1/2'] = X['runtimeMinutes'] ** 1/2
        #res['1/runtimeMinutes'] = 1/X['runtimeMinutes']

        res['knn_startYear'] = self.knn_year.predict(X[['endYear']])
        res['knn_runtimeMinutes'] = self.knn_min.predict(X[['runtimeMinutes']])
        #res['knn_titleLength'] = self.knn_len.predict(X[['titleLength']])
        #res['knn_numVotes'] = self.knn_vot.predict(X[['numVotes']])

        return res

In [1927]:
rmses = []
pr = 0
for _ in range(50):
    X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.2)
    prep = Preprocessing().fit(X_train, y_train)
    X_train_new = prep.transform(X_train)
    X_train_new = X_train_new.fillna(X_train_new.mean())
    X_test = prep.transform(X_test)
    model = LR().fit(X_train_new, y_train)
    preds = model.predict(X_test)
    rmses.append(mse(y_test, preds) ** 0.5)
    preds = model.predict(prep.transform(test))
    pr += preds

print(f'{np.mean(rmses):.2f} +/- {np.std(rmses):.2f}')


1.19 +/- 0.05


In [1928]:
train

,endYear,runtimeMinutes,numVotes,titleLength,genderVoters,worldPromotion,Drama,Horror,Short,Action,...,short,movie,tvMovie,tvEpisode,tvSeries,tvMiniSeries,video,videoGame,tvSpecial,tvShort
Id,,,,,,,,,,,,,,,,,,,,,
0,1997.622222,23.000000,32,14,17.0,1,1,1,1,0,...,1,0,0,0,0,0,0,0,0,0
1,1997.622222,112.000000,13110,29,6513.0,0,1,0,0,1,...,0,1,0,0,0,0,0,0,0,0
2,1997.622222,7.000000,5,10,2.0,1,1,0,1,0,...,1,0,0,0,0,0,0,0,0,0
3,1997.622222,58.572773,6,18,1.0,1,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,1997.622222,58.572773,10,9,7.0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1997.622222,58.572773,19,31,9.0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1996,1997.622222,20.000000,10,14,9.0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
1997,1997.622222,44.000000,10,11,1.0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [1935]:
# preds = model.predict(prep.transform(test))
prep = Preprocessing().fit(X_train, y_train)
train_new = prep.transform(train)    
train_new = train_new.fillna(train_new.mean())
test = prep.transform(test)
model = LR().fit(train_new, target)
preds = model.predict(test)
predicted_df = pd.DataFrame(preds)
predicted_df.to_csv('Films.csv', index=False,  header=False)